In [1]:
import pandas as pd
import requests
from io import StringIO

class AtomicDataLoader:
    def __init__(self, source):
        self.source = source

    def load_data(self):
        if self.source.startswith('http://') or self.source.startswith('https://'):
            response = requests.get(self.source)
            lines = response.text.split('\n')
        else:
            with open(self.source, 'r') as file:
                lines = file.readlines()
        
        data = []
        for line in lines[1:]:  # Skip the first line
            if line.strip():  # Skip empty lines
                parts = line.split()
                element = parts[0]
                index = int(parts[1])
                x_coord = float(parts[2])
                y_coord = float(parts[3])
                z_coord = float(parts[4])
                data.append([element, index, x_coord, y_coord, z_coord])
        
        df = pd.DataFrame(data, columns=['Element', 'Index', 'X', 'Y', 'Z'])
        return df

# # Usage example with a file path
# file_path = '/mnt/data/sio2-cart.chem3d'
# loader = AtomicDataLoader(file_path)
# df = loader.load_data()

# Usage example with a URL
url = 'https://isaacs.sourceforge.io/tests/sio2-cart.chem3d'
loader = AtomicDataLoader(url)
df = loader.load_data()
print(f"Number of rows: {len(df)}, number of silicon atoms: {len(df[df['Element'] == 'Si'])}, number of oxygen atoms: {len(df[df['Element'] == 'O'])}")
df.head()

Number of rows: 3000, number of silicon atoms: 1000, number of oxygen atoms: 2000


,Element,Index,X,Y,Z
0,O,1,5.043,-16.116,-4.787
1,O,2,7.184,-16.066,-3.850
2,O,3,-8.529,-1.029,-0.326
3,O,4,2.347,5.238,-6.673
4,O,5,-3.873,13.915,7.447


In [2]:
# reduce the number of points by only keeping atoms
# within a certain distance of the origin
df['Distance'] = (df['X']**2 + df['Y']**2 + df['Z']**2)**0.5
df = df[df['Distance'] < 8]
print(f"Number of rows: {len(df)}, number of silicon atoms: {len(df[df['Element'] == 'Si'])}, number of oxygen atoms: {len(df[df['Element'] == 'O'])}")
# use plotly to visualize the data. Oxygen atoms are red, silicon atoms are blue
import plotly.express as px
fig = px.scatter_3d(df, x='X', y='Y', z='Z', color='Element', title='SiO2')
fig.update_traces(marker_size = 3)
fig.show()

Number of rows: 140, number of silicon atoms: 45, number of oxygen atoms: 95


In [6]:
# keep all atoms in the upper row
points_upper = df[['X', 'Y', 'Z']].values
# remove silicon atoms in the lower row
deletion_list = df[df['Element'] == 'Si'].index
# Get one-parameter persistence information for the upper row
from commutazzio.persistence import PD_Points3D
from cpes import Points3D
SiO2_persistence=PD_Points3D(Points3D(points_upper))
# get the maximum critical radius when homology dimension is one
max_radius = max([d for (b,d,dim) in SiO2_persistence.diagram_1_r])
print(f"Max radius: {max_radius}")
# show the persistence diagrams at dimension one before and after thinning
SiO2_persistence.plot_1D(plotrange=(0,4))
OxygenOnly_persistence=PD_Points3D(Points3D(df[df['Element'] == 'O'][['X', 'Y', 'Z']].values))
OxygenOnly_persistence.plot_1D(plotrange=(0,4))

Alpha complex is of dimension 3
The radius is squared.
Max radius: 10.395887562852284
Alpha complex is of dimension 3
The radius is squared.


In [5]:
SiO2_persistence.plot_1D(plotrange=(0,4))

In [4]:
from commutazzio.filtration import pointCloud2Filtration, points_to_clfiltration_chro
# plot radii for the connected persistence diagram
# from zero to max_radius in 50 steps
plot_radii = [i*max_radius/50 for i in range(51)]
# 0 for atoms not in the deletion_list, 1 for atoms in the deletion_list
# labels = [0 if i in deletion_list else 1 for i in range(len(points_upper))]
# filtration=points_to_clfiltration_chro(points_upper, labels, 2, plot_radii)
filtration=pointCloud2Filtration(points_upper, deletion_list, plot_radii,max_simplex_dim=2)

Creating filtration...
